# Big Red Soundscape
Notebook Playground 2  
_by Ahmed Sultan AND CARLY HU, for Cornell Data Journal_

In [2]:
import json
import math

import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# update access keys here for Spotify API here :) 
client_id = '860ada55cc094dbf906250fe2d4caf48'
client_secret = '75fe4137c4a54541800d19b74dc706fb'

# sp is how we interface with the Spotify API
# read the docs here: https://spotipy.readthedocs.io/en/2.22.1/
sp = spotipy.Spotify(client_credentials_manager=\
                     SpotifyClientCredentials(client_id, client_secret))

In [4]:
# Big Data, Machine Learning...
# https://www.youtube.com/watch?v=bqqCTC9nQDY

# let's aggregate an entire user's public library of songs
playlists = []
LIMIT = 20
selected_user = 'carlatude'

query = sp.user_playlists(selected_user, limit=LIMIT)

n = 0

while len(query['items']) != 0:
    n += LIMIT
    for i in query['items']:
        playlists.append({'title': i['name'],'id': i['id'], 'length': i['tracks']['total']})
    if LIMIT == len(query['items']):
        query = sp.user_playlists(selected_user, limit=LIMIT, offset=n)
    else:
        break
        
# there's definitely a cleaner way to do this that we'll figure out
# im definitely lazy

In [5]:
playlists

[{'title': 'spring 23', 'id': '0AberywlM7KTsCBRAs1Hs0', 'length': 46},
 {'title': 'bouncing', 'id': '7lspAgfh11vTTTpm5G1wkf', 'length': 12},
 {'title': 'kenny', 'id': '4jX7u824J8LFH7iYP6d44G', 'length': 9},
 {'title': 'walking energetically',
  'id': '09EmE7ZmePNp8qtvzePdWE',
  'length': 16},
 {'title': 'instrumental', 'id': '42DYI4rZtxyySmX3sjAoBy', 'length': 40},
 {'title': 'pretty', 'id': '2EXVB9JsrIPINY25rJ7Bth', 'length': 65},
 {'title': 'comfort', 'id': '7645opI5xNijOpeaQ6wOJK', 'length': 41},
 {'title': 'screaming crying', 'id': '2C1QFkfhQDDw0jYm5Tr9pE', 'length': 34},
 {'title': 'slay pt 2', 'id': '2U9jxw1KdLtCATpYQZC0qv', 'length': 47},
 {'title': 'taylor (sad)', 'id': '5WCNl7XT1KuRN2nLyhtmBH', 'length': 51},
 {'title': 'agh!!!!', 'id': '0zefXRE0Pr0W72A5966PLn', 'length': 27},
 {'title': 'wake up', 'id': '0DN6Ihs7y7XarkOgXpMkau', 'length': 19},
 {'title': 'groovy', 'id': '7oKjtKN4PQJ1ZA6n5N9qfN', 'length': 118},
 {'title': 'bedroom pop', 'id': '6QGojHMaB0ktJ32vpsxRZk', 'length

In [6]:
# number of public playlists
print(len(playlists))

29


In [ ]:
# making sure we can get all songs in a playlist rather than just the first 100
# https://stackoverflow.com/questions/55690063/is-there-a-method-to-retrieve-all-tracks-from-a-playlist-using-spotipy 

def user_playlist_tracks_full(user, playlist_id=None, fields=None):
    """ Get full details of the tracks of a playlist owned by a user.
        https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/

        Parameters:
            - user - the id of the user
            - playlist_id - the id of the playlist
            - fields - which fields to return
    """

    # first run through also retrieves total no of songs in library
    response = sp.user_playlist_tracks(user, playlist_id, fields=None, limit=100)
    results = response["items"]

    # subsequently runs until it hits the user-defined limit or has read all songs in the library
    while len(results) < response["total"]:
        response = sp.user_playlist_tracks(
            user, playlist_id, fields=fields, limit=100, offset=len(results)
        )
        results.extend(response["items"])
    
    return results
        

In [ ]:
# now, let's go through each playlist
# this will basically be the same as before

songs = []

for p in playlists:
    
    results = sp.playlist(p['id'])
    
    extended_results = user_playlist_tracks_full(user = selected_user, playlist_id = results['id']) # more than 100
    
    for r in extended_results:
        songs.append(r['track']['id']) 

In [47]:
def get_tracks_full(songs):
    results = []
    while len(results) < len(songs):
        response = sp.tracks(songs[len(results):len(results) + 50])
        results.extend(response)
    return results

In [ ]:
song_info = {'id': [], 'title': [], 'artist': [], 'album': [], 
             'explicit': [], 'popularity': []}

songs_meta = get_tracks_full(songs)

In [37]:
print(len(songs)) # total songs

1887


In [38]:
def get_audio_features_full(songs):
    results = []
    while len(results) < len(songs):
        response = sp.audio_features(songs[len(results):len(results) + 100])
        results.extend(response)
    return results

In [39]:
len(get_audio_features_full(songs))

1887

In [42]:
print(get_audio_features_full(songs[0]))

[{'danceability': 0.467, 'energy': 0.624, 'key': 10, 'loudness': -8.511, 'mode': 0, 'speechiness': 0.0508, 'acousticness': 0.143, 'instrumentalness': 0.000899, 'liveness': 0.145, 'valence': 0.31, 'tempo': 141.889, 'type': 'audio_features', 'id': '6BQrmjAgPfPTwILoFEK3Hd', 'uri': 'spotify:track:6BQrmjAgPfPTwILoFEK3Hd', 'track_href': 'https://api.spotify.com/v1/tracks/6BQrmjAgPfPTwILoFEK3Hd', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6BQrmjAgPfPTwILoFEK3Hd', 'duration_ms': 225947, 'time_signature': 4}, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [43]:
features_df = pd.DataFrame.from_dict(get_audio_features_full(songs))